# The problem 
is about, finding most popular marvel character regarding Hero Graph data (which heros seen together in movies). 
On the other hand, we need hero-name in the result. Instead of joining, we will broadcast hero-names to all executers.

So what's the definition of **Popular marvel character regarding rating data ?**

#### steps
* load hero data and convert into dictionary then broadcast
* load graph data, use every lines & colums into 1 to define **popular heros**


In [1]:
sc.version

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1555140556054_0003,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.
u'2.4.0'

### Hero master data
marvel-hero-id hero-name

In [21]:
heroLines = sc.textFile("s3://spark-dataset-sandbox/datasets/marvel/marvel-hero-names.txt")

VBox()

In [23]:
heroLines.take(2)

VBox()

[u'1 "24-HOUR MAN/EMMANUEL"', u'2 "3-D MAN/CHARLES CHAN"']

#### create heros dictionary to query id based

In [60]:
heros = heroLines.map(lambda lines: lines.split(" ")) \
                 .map(lambda x: (int(x[0]), x[1])) \
                 .collectAsMap()

VBox()

In [61]:
heros

VBox()

{1: u'"24-HOUR', 2: u'"3-D', 3: u'"4-D', 4: u'"8-BALL/"', 5: u'"A"', 6: u'"A\'YIN"', 7: u'"ABBOTT,', 8: u'"ABCISSA"', 9: u'"ABEL"', 10: u'"ABOMINATION/EMIL', 11: u'"ABOMINATION', 12: u'"ABOMINATRIX"', 13: u'"ABRAXAS"', 14: u'"ADAM', 15: u'"ABSALOM"', 16: u'"ABSORBING', 17: u'"ABSORBING', 18: u'"ACBA"', 19: u'"ACHEBE,', 20: u'"ACHILLES"', 21: u'"ACHILLES', 22: u'"ACROBAT/CARL', 23: u'"ADAM', 24: u'"ADAMS,', 25: u'"ADAMS,', 26: u'"ADAMS,', 27: u'"ADAMS,', 28: u'"ADAMS,', 29: u'"ADAMSON,', 30: u'"ADAMSON,', 31: u'"ADMIRAL', 32: u'"ADORA"', 33: u'"ADORA', 34: u'"ADRIA"', 35: u'"ADVA"', 36: u'"ADVENT/KYLE', 37: u'"ADVERSARY"', 38: u'"AEGIS/TREY', 39: u'"AENTAROS"', 40: u'"AFTERLIFE"', 41: u'"AGAMEMNON"', 42: u'"AGAMEMNON', 43: u'"AGAMEMNON', 44: u'"AGAMOTTO"', 45: u'"AGARN,', 46: u'"AGED', 47: u'"AGEE,', 48: u'"AGEE,', 49: u'"AGENT', 50: u'"AGENT', 51: u'"AGENT-X/"', 52: u'"AGGAMON"', 53: u'"AGINAR"', 54: u'"AGON"', 55: u'"AGONY/"', 56: u'"AGUIRRE,', 57: u'"AHAB/RORY', 58: u'"AHQLAU"', 59: 

In [62]:
herosDictionary = sc.broadcast(heros)

VBox()

### Hero's graph data (which heros seen together in movies)
marvel-hero-id marvel-hero-id marvel-hero-id marvel-hero-id ....

In [63]:
heroGraphLines = sc.textFile("s3://spark-dataset-sandbox/datasets/marvel/marvel-graph.txt")

VBox()

In [64]:
heroGraphLines.take(2)

VBox()

[u'5988 748 1722 3752 4655 5743 1872 3413 5527 6368 6085 4319 4728 1636 2397 3364 4001 1614 1819 1585 732 2660 3952 2507 3891 2070 2239 2602 612 1352 5447 4548 1596 5488 1605 5517 11 479 2554 2043 17 865 4292 6312 473 534 1479 6375 4456 ', u'5989 4080 4264 4446 3779 2430 2297 6169 3530 3272 4282 6432 2548 4140 185 105 3878 2429 1334 4595 2767 3956 3877 4776 4946 3407 128 269 5775 5121 481 5516 4758 4053 1044 1602 3889 1535 6038 533 3986 ']

In [65]:
herosGraph = heroGraphLines.flatMap(lambda lines: lines.split()) \
                           .cache() \
                           .map(lambda x: (int(x), 1)) \
                           .reduceByKey(lambda x,y: (x+y)) \
                           .cache() \
                           .map(lambda x: (x[1],x[0])) \
                           .max()

VBox()

In [66]:
herosGraph

VBox()

(1937, 859)

In [68]:
herosDictionary.value[herosGraph[1]], herosGraph[0]

VBox()

(u'"CAPTAIN', 1937)